# Overview
Notebook performing homolog group inferences for mobile elements in PCG, MAGs and metagenome assembled contigs of Archaea containing FsxAs.

___
Workflow of the analysis is as follows (please refere to Suppelementary Figures in order to see a detailed, graphic representation of this pipeline):
- Infer ORFs for contigs containing FsxA with *getorf*.
- Subsetting annotated features in those contigs.
- Stablishing a correlation between predicted ORFs and annotated features (employed later in the analysis).
- Infer groups of homologous sequences with *get_homologues*, using predicted ORFs as input sequences.
- Subsetting homolog groups containing at least one annotated feature.
- Perform remote homologs searches for each homolog group: HMM profiles are created by employing their sequences as queries for a jackhmmer search against UniRef50. HMM profiles are thereafter created, and homolog groups are collapsed by performing pairwise comparisons between them with hhalign.
- Filter homolog groups in order to contemplate only sequences present in MEs of a reasonable size
___

# Infer ORFs for contigs containing FsxA with *getorf*

## Subsetting mobile elements with hits of FsxA in MAGs

In [1]:
# loading libraries
import os
import subprocess
import pandas as pd
import glob
import re
from Bio import SeqIO

# load table listing hits of FsxA in MAGs
fsxA_hits_in_MAGs_table = pd.read_csv('../data/MAGS/fsxa.MAGS.tab', sep = '\t', names=['GFF file', 'Contig', 'Annotation source', 'Feature type', 'Start', 'End', 'extra', 'Strand', 'E-value', 'ID'])
fsxA_hits_in_MAGs_table
contigs_with_fsxA_hits = fsxA_hits_in_MAGs_table['Contig'].to_list()

# create dictionary with all MAG contigs
genome_fna_files = [fna_file for fna_file in glob.glob('../data/MAGS/ftp.ncbi.nlm.nih.gov/*/*/*/*/*/*/*/*v*_genomic.fna') if not re.match(pattern = '.*rna_from.*|.*cds_from.*', string = fna_file)]

# subset contigs of MAGs containing hits of FsxA
target_contigs_MAGs = []
for fna_file in genome_fna_files:
    for record in SeqIO.parse(fna_file, 'fasta'):
        if record.id in contigs_with_fsxA_hits:
            # get MAG tag
            mag_tag = fna_file.split('/')[10].split('_')[1]
            target_contigs_MAGs.append((mag_tag, record))

# saving contigs of MAGs with hits of FsxA as ME with their code
for mag_contig_set in target_contigs_MAGs:
    # depacking variables
    mag_tag, sequence_record = mag_contig_set
    # setting output file name
    output_file = '../data/mobile_elements_sequences/ME.{0}.MAG.fna'.format(mag_tag)
    if not os.path.exists(output_file):
        # saving contig sequence
        with open(output_file, 'w') as output_handle:
            SeqIO.write(sequence_record, output_handle, "fasta")

## Formating mobile elements with hits of FsxA in metagenomes
Contigs with hits for FsxA in metagenomes are moved to a common folder with those of PCG Archaea and MAGs.

In [2]:
# import library
import shutil

# list metagenome contig with hits in FsxA (previously subsetted by Dr. Hector Romero)
fsxA_hits_in_metagenomes = glob.glob('../data/mobile_elements_sequences_metagenomes/*fasta')

# moving these files to common folder with PCG and MAGs Archaea contigs with hits in FsxA
for contig_file in fsxA_hits_in_metagenomes:
    # getting tag of metagenome
    metagenome_tag = contig_file.split('/')[3].replace('.fasta', '')
    # create output file name
    output_file = '../data/mobile_elements_sequences/ME.{0}.metagenome.fna'.format(metagenome_tag)
    # moving
    if not os.path.exists(output_file):
        shutil.copy(src = contig_file, dst = output_file)

## Infering ORFs with *getorf*.
Script from previous analyses is employed.

In [3]:
%run getting_orfs.py

# Subsetting annotated features in those contigs

## Running for MAGs

In [4]:
# creating dictionary with protein annotated for each MAG
MAG_proteins_dict = {}
for faa_file in glob.glob('../data/MAGS/*/*/*/*/*/*/*/*/*translated_cds.faa'):
    # reading proteins
    faa_seqs = SeqIO.parse(faa_file, 'fasta')
    # adding to dictionary every protein
    for record in faa_seqs:
        # getting protein name
        proteinid = [string.replace('[', '').replace(']', '').replace('protein_id=', '') for string in record.description.split(' ') if re.findall(pattern = 'protein_id', string = string)]
        try:
            proteinid = proteinid[0]
            MAG_proteins_dict.update({proteinid: record})
        except:
            #print(record)
            locus_tag = [string.replace('[', '').replace(']', '').replace('locus_tag=', '') for string in record.description.split(' ') if re.findall(pattern = 'locus_tag', string = string)]
            locus_tag = locus_tag[0]
            MAG_proteins_dict.update({locus_tag: record})
        

# list all MAG contigs
mag_gff_files = glob.glob('../data/MAGS/ftp.ncbi.nlm.nih.gov/*/*/*/*/*/*/*/*v*gff')

# for each MAG
for mag_gff_file in mag_gff_files:
    # get MAG tag
    mag_tag = mag_gff_file.split('/')[10].split('_')[1]
    
    # charge annotation file
    mag_gff = pd.read_csv(mag_gff_file, 
                          sep = '\t', 
                          comment = '#',
                          names=['Contig', 'Annotation source', 'Feature type', 'Start', 'End', 'extra', 'Strand', 'E-value', 'ID'])
    mag_gff
    # get mag target contig
    mag_ME_fasta_file = '../data/mobile_elements_sequences/ME.{0}.MAG.fna'.format(mag_tag)
    mag_target_contig = [record.id for record in SeqIO.parse(mag_ME_fasta_file, 'fasta')]
    # subset for target contig
    mag_target_contig
    # get features of the contig
    mag_ME_annotated_proteins_table = mag_gff.query("`Contig` in @mag_target_contig & `Feature type` == 'CDS'")
    mag_ME_annotated_proteins_IDs = [feature.split(';')[0].replace('ID=cds-', '') for feature in mag_ME_annotated_proteins_table['ID'].to_list()]
    
    # subsetting proteins
    mag_ME_protein_seqs = [MAG_proteins_dict[protein_id] for protein_id in mag_ME_annotated_proteins_IDs]
    
    # modifying ID and description in order to remove white spaces (disturbs future analyses)
    for record in mag_ME_protein_seqs:
        record.id = record.id.replace(' ', '_')
        record.description = record.description.replace(' ', '_')
    
    # checking that number of extracted sequences is OK
    if mag_ME_annotated_proteins_table.shape[0] != len(mag_ME_protein_seqs):
        print('Problem extracting sequences of MAG:', mag_tag)
    elif mag_ME_annotated_proteins_table.shape[0] == len(mag_ME_protein_seqs):
        print('Sequence extraction went OK for MAG:', mag_tag)
    # save to file
    output_faa_file = '../results/ME_PATRIC_annotated_features/ME.{0}.PATRIC_annotated_features.faa'.format(mag_tag)
    if not os.path.exists(output_faa_file):
        with open(output_faa_file, 'w') as output_handle:
            SeqIO.write(mag_ME_protein_seqs, output_handle, "fasta")

Sequence extraction went OK for MAG: 003641755.1
Sequence extraction went OK for MAG: 003644925.1
Sequence extraction went OK for MAG: 003661485.1
Sequence extraction went OK for MAG: 003662005.1
Sequence extraction went OK for MAG: 003662775.1
Sequence extraction went OK for MAG: 013152655.1
Sequence extraction went OK for MAG: 012026795.1
Sequence extraction went OK for MAG: 012962785.1
Sequence extraction went OK for MAG: 000830315.1
Sequence extraction went OK for MAG: 011044075.1
Sequence extraction went OK for MAG: 011380245.1
Sequence extraction went OK for MAG: 011042275.1
Sequence extraction went OK for MAG: 011039335.1
Sequence extraction went OK for MAG: 011358415.1


## Running for metagenomes

In [6]:
# listing metagenome GFF files
metagenome_gff_files = glob.glob('../data/metagenome_annotations/*csv')

# for each gff file perform pipeline
for metagenome_gff_file in metagenome_gff_files:
    # loading metagenome contig GFF
    metagenome_gff_table = pd.read_csv(metagenome_gff_file, sep = '\t')
    # getting metagenome tag
    metagenome_tag = metagenome_gff_file.split('/')[3].replace('.csv', '').lower().split('.')[0]
    # create dictionary to allocate target sequences
    metagenome_cds_seqs = []
    metagenome_aa_seqs = []
    # loading fna file
    metagenome_contig_dna_file = '../data/mobile_elements_sequences/ME.{0}.metagenome.fna'.format(metagenome_tag)
    metagenome_contig_dna = [record for record in SeqIO.parse(metagenome_contig_dna_file, 'fasta')]
    metagenome_contig_dna = metagenome_contig_dna[0]
    # creating dictionaries to allocate sequences
    metagenoma_CDS_seqs = []
    metagenoma_AA_seqs = []
    # extracting CDS sequence for each feature in target contig
    for index, row in metagenome_gff_table.iterrows():
        # set variables
        gene_id = row['Gene ID']
        gene_name = row['Gene Product Name']
        start = row['Start Coord']
        end = row['End Coord']
        strand = row['Strand']
        # getting sequence and storing it in corresponding dictionary
        cds_seq = metagenome_contig_dna[start-1:end]
        # taking into account strandedness
        if strand == '-':
            cds_seq = cds_seq.reverse_complement()
        cds_seq.id = gene_id
        cds_seq.description = gene_name 
        metagenoma_CDS_seqs.append(cds_seq)
        # translating sequence and storing it in corresponding dictionary
        aa_seq = SeqIO.SeqRecord(seq = cds_seq.seq.translate(table = 11), 
                           id = gene_id.replace(' ', '_'),
                           description = gene_name.replace(' ', '_'))
        metagenoma_AA_seqs.append(aa_seq)
    # checking that numbers of extracted seqs are OK
    if metagenome_gff_table.shape[0] != len(metagenoma_AA_seqs):
        print('Problem extracting sequences of MAG:', metagenome_tag)
    elif metagenome_gff_table.shape[0] == len(metagenoma_AA_seqs):
        print('Sequence extraction went OK for MAG:', metagenome_tag)
    # saving protein FASTA file
    output_faa_file = '../results/ME_PATRIC_annotated_features/ME.{0}.PATRIC_annotated_features.faa'.format(metagenome_tag)
    if not os.path.exists(output_faa_file):
        with open(output_faa_file, 'w') as output_handle:
            SeqIO.write(metagenoma_AA_seqs, output_handle, "fasta")

Sequence extraction went OK for MAG: ga0075122_10000827
Sequence extraction went OK for MAG: ga0207733_100382
Sequence extraction went OK for MAG: ga0207718_100100
Sequence extraction went OK for MAG: ga0078972_1006503
Sequence extraction went OK for MAG: ga0075120_10000694
Sequence extraction went OK for MAG: ga0065719_100268
Sequence extraction went OK for MAG: ga0172377_10000119
Sequence extraction went OK for MAG: ga0172379_10000243
Sequence extraction went OK for MAG: ga0172379_10001592
Sequence extraction went OK for MAG: ga0207715_100102
Sequence extraction went OK for MAG: ga0075122_10000557
Sequence extraction went OK for MAG: ga0222667_1000160
Sequence extraction went OK for MAG: ga0222658_1000332
Sequence extraction went OK for MAG: ga0164313_10000250
Sequence extraction went OK for MAG: ga0075122_10000851


/home/mauri/anaconda3/envs/r-environment/lib/python3.9/site-packages/Bio/Seq.py:2334: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Sequence extraction went OK for MAG: ga0208375_1000150
Sequence extraction went OK for MAG: ga0116200_10000074
Sequence extraction went OK for MAG: jgi12330j12834_1000008
Sequence extraction went OK for MAG: ga0207719_100190


## Running for PCG

### First locating ME in genomes

In [9]:
# importing libraries
import os
import subprocess

# getting MEs coordinates in genomes in order to parse coding sequences in them
# creating list with 2-uple storing ME file and genome file
me_data = [(ME.replace('ME.', '').replace('.fna', ''),
            '../data/mobile_elements_sequences/{0}'.format(ME), 
            '../data/genome_annotations/{0}/{0}.fna'.format(ME.replace('ME.', '').replace('.fna', ''))) for ME in os.listdir('../data/mobile_elements_sequences/')]

# creating some directories to store results and databases
for directory in ['../data/genome_blastdbs', '../results/MEs_against_genomes']:
  if not os.path.exists(directory):
    os.mkdir(directory)

# creating BLAST nucleotidic database for each genome
for data in me_data:
  # unpacking variables
  genome_tag, mobile_element, genome = data
  # create BLAST database
  make_blastdb_command = 'makeblastdb -in {0} -dbtype nucl -out ../data/genome_blastdbs/{1}_db -parse_seqids'.format(genome, genome_tag).split(' ')
  subprocess.run(make_blastdb_command)
  # BLASTN ME against the respective genome
  blastn_command = "blastn -db ../data/genome_blastdbs/{0}_db -query {1} -out ../results/MEs_against_genomes/ME.{0}_vs_{0}.blout -evalue 1e-5 -num_threads 1 -outfmt 6".format(genome_tag, mobile_element).split(' ')
  subprocess.run(blastn_command)

### Extracting annotated features

In [10]:
# creating dictionary with protein annotated for each MAG
import gzip

halovivax_proteins_dict = {}
for faa_file in glob.glob('../data/halovivax/GCF_017357405.1_ASM1735740v1_translated_cds.faa.gz'):
    # reading proteins
    #faa_seqs = SeqIO.parse(faa_file, 'fasta')
    #faa_seqs = []
    with gzip.open(faa_file, "rt") as handle:
        faa_seqs = [record for record in SeqIO.parse(handle, "fasta")]
    # adding to dictionary every protein
    for record in faa_seqs:
        # getting protein name
        proteinid = [string.replace('[', '').replace(']', '').replace('protein_id=', '') for string in record.description.split(' ') if re.findall(pattern = 'protein_id', string = string)]
        try:
            proteinid = proteinid[0]
            halovivax_proteins_dict.update({proteinid: record})
        except:
            #print(record)
            locus_tag = [string.replace('[', '').replace(']', '').replace('locus_tag=', '') for string in record.description.split(' ') if re.findall(pattern = 'locus_tag', string = string)]
            locus_tag = locus_tag[0]
            halovivax_proteins_dict.update({locus_tag: record})

In [11]:
# getting contig hit and start and end of mobile element
blastn_outfmt_columns = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
halovivax_blast_hit = pd.read_csv('../results/MEs_against_genomes/ME.017357405.1_vs_017357405.1.blout', sep = '\t', names = blastn_outfmt_columns)

halovivax_blast_hit

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,NZ_CP071597.1,NZ_CP071597.1,100.0,94500,0,0,1,94500,3085001,3179500,0.0,174500.0


In [12]:
mag_gff_file = '../data/genome_annotations/017357405.1/017357405.1.gff'
mag_tag = mag_gff_file.split('/')[3]    
# charge annotation file
mag_gff = pd.read_csv(mag_gff_file, 
                      sep = '\t', 
                      comment = '#',
                      names=['Contig', 'Annotation source', 'Feature type', 'Start', 'End', 'extra', 'Strand', 'E-value', 'ID'])
mag_gff
# get mag target contig
mag_ME_fasta_file = '../data/mobile_elements_sequences/ME.{0}.fna'.format(mag_tag)
mag_target_contig = halovivax_blast_hit['sseqid'].to_list()
mag_target_start = halovivax_blast_hit['sstart'].to_list()[0]
mag_target_end = halovivax_blast_hit['send'].to_list()[0]
# subset for target contig
mag_target_contig
# get features of the contig
mag_ME_annotated_proteins_table = mag_gff.query("`Contig` in @mag_target_contig & `Feature type` == 'CDS' & Start >= @mag_target_start & End <= @mag_target_end")

mag_ME_annotated_proteins_table.head()

,Contig,Annotation source,Feature type,Start,End,extra,Strand,E-value,ID
5825,NZ_CP071597.1,Protein Homology,CDS,3085325,3085912,.,-,0,ID=cds-WP_207587062.1;Parent=gene-J1N62_RS1446...
5827,NZ_CP071597.1,GeneMarkS-2+,CDS,3085939,3086346,.,-,0,ID=cds-WP_207587064.1;Parent=gene-J1N62_RS1447...
5829,NZ_CP071597.1,GeneMarkS-2+,CDS,3086605,3086787,.,+,0,ID=cds-WP_207587066.1;Parent=gene-J1N62_RS1447...
5831,NZ_CP071597.1,GeneMarkS-2+,CDS,3086855,3088243,.,-,0,ID=cds-WP_207587068.1;Parent=gene-J1N62_RS1448...
5833,NZ_CP071597.1,GeneMarkS-2+,CDS,3088236,3088883,.,-,0,ID=cds-WP_207587070.1;Parent=gene-J1N62_RS1448...


In [13]:
# list all MAG contigs
halovivax_gff_files = glob.glob('../data/genome_annotations/017357405.1/017357405.1.gff')

# for each MAG
for mag_gff_file in halovivax_gff_files:
    # get MAG tag
    mag_tag = mag_gff_file.split('/')[3]
    
    # charge annotation file
    mag_gff = pd.read_csv(mag_gff_file, 
                          sep = '\t', 
                          comment = '#',
                          names=['Contig', 'Annotation source', 'Feature type', 'Start', 'End', 'extra', 'Strand', 'E-value', 'ID'])
    mag_gff
    # get mag target contig
    mag_ME_fasta_file = '../data/mobile_elements_sequences/ME.{0}.fna'.format(mag_tag)
    mag_target_contig = halovivax_blast_hit['sseqid'].to_list()
    mag_target_start = halovivax_blast_hit['sstart'].to_list()[0]
    mag_target_end = halovivax_blast_hit['send'].to_list()[0]
    # subset for target contig
    mag_target_contig
    # get features of the contig
    mag_ME_annotated_proteins_table = mag_gff.query("`Contig` in @mag_target_contig & `Feature type` == 'CDS' & Start >= @mag_target_start & End <= @mag_target_end")
    mag_ME_annotated_proteins_IDs = [feature.split(';')[0].replace('ID=cds-', '') for feature in mag_ME_annotated_proteins_table['ID'].to_list()]
    
    # subsetting proteins
    mag_ME_protein_seqs = [halovivax_proteins_dict[protein_id] for protein_id in mag_ME_annotated_proteins_IDs]
    
    # modifying ID and description in order to remove white spaces (disturbs future analyses)
    for record in mag_ME_protein_seqs:
        record.id = record.id.replace(' ', '_')
        record.description = record.description.replace(' ', '_')
    
    # checking that number of extracted sequences is OK
    if mag_ME_annotated_proteins_table.shape[0] != len(mag_ME_protein_seqs):
        print('Problem extracting sequences of MAG:', mag_tag)
    elif mag_ME_annotated_proteins_table.shape[0] == len(mag_ME_protein_seqs):
        print('Sequence extraction went OK for MAG:', mag_tag)
    # save to file
    output_faa_file = '../results/ME_PATRIC_annotated_features/ME.{0}.PATRIC_annotated_features.faa'.format(mag_tag)
    if not os.path.exists(output_faa_file):
        with open(output_faa_file, 'w') as output_handle:
            SeqIO.write(mag_ME_protein_seqs, output_handle, "fasta")

Sequence extraction went OK for MAG: 017357405.1


In [14]:
mag_ME_annotated_proteins_table

,Contig,Annotation source,Feature type,Start,End,extra,Strand,E-value,ID
5825,NZ_CP071597.1,Protein Homology,CDS,3085325,3085912,.,-,0,ID=cds-WP_207587062.1;Parent=gene-J1N62_RS1446...
5827,NZ_CP071597.1,GeneMarkS-2+,CDS,3085939,3086346,.,-,0,ID=cds-WP_207587064.1;Parent=gene-J1N62_RS1447...
5829,NZ_CP071597.1,GeneMarkS-2+,CDS,3086605,3086787,.,+,0,ID=cds-WP_207587066.1;Parent=gene-J1N62_RS1447...
5831,NZ_CP071597.1,GeneMarkS-2+,CDS,3086855,3088243,.,-,0,ID=cds-WP_207587068.1;Parent=gene-J1N62_RS1448...
5833,NZ_CP071597.1,GeneMarkS-2+,CDS,3088236,3088883,.,-,0,ID=cds-WP_207587070.1;Parent=gene-J1N62_RS1448...
...,...,...,...,...,...,...,...,...,...
6039,NZ_CP071597.1,Protein Homology,CDS,3175529,3176281,.,+,0,ID=cds-WP_207587203.1;Parent=gene-J1N62_RS1500...
6041,NZ_CP071597.1,GeneMarkS-2+,CDS,3176288,3176635,.,+,0,ID=cds-WP_207587205.1;Parent=gene-J1N62_RS1500...
6043,NZ_CP071597.1,GeneMarkS-2+,CDS,3176846,3177706,.,+,0,ID=cds-WP_207587207.1;Parent=gene-J1N62_RS1501...
6045,NZ_CP071597.1,GeneMarkS-2+,CDS,3178015,3178596,.,+,0,ID=cds-WP_207587209.1;Parent=gene-J1N62_RS1501...


In [15]:
# for the other PCGs
pcg_list = [filename.rpartition('/')[2].replace('ME.', '').replace('.fna', '') for filename in glob.glob('/home/mauri/fusogenos/fusexins/FsxA/analysis_mobile_elements/data/mobile_elements_sequences/*.fna')] # create list of PCG IDs by parsing first analyses
pcg_gffs = ['../data/genome_annotations/{0}/{0}.PATRIC.gff'.format(genome_id) for genome_id in pcg_list] # create path of GFFs
pcg_blast_files = ['../results/MEs_against_genomes/ME.{0}_vs_{0}.blout'.format(genome_id) for genome_id in pcg_list]
pcg_blast_dict = {} # create dictionary of BLASTN results

for blast_file_pack in zip(pcg_list, pcg_blast_files):
    genome_id, blast_file = blast_file_pack # depack variables
    blastn_outfmt_columns = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore'] # set column names
    blast_hit_table = pd.read_csv(blast_file, sep = '\t', names = blastn_outfmt_columns).query("`mismatch` == 0 & `evalue` == 0") # get table
    pcg_blast_dict.update({genome_id: blast_hit_table}) # update

In [16]:
# creating a dictionary with PCG proteins
pcg_protein_dict = {}

for protein_fasta in ['../data/genome_annotations/{0}/{0}.PATRIC.faa'.format(genome_id) for genome_id in pcg_list]: # iterate over files of proteins
    genome_id = protein_fasta.split('/')[3] # get genome_id
    genome_records = [record for record in SeqIO.parse(protein_fasta, 'fasta')] # load records
    pcg_protein_dict.update({genome_id: {}}) # intialize key
    for record in genome_records:
        pcg_protein_dict[genome_id].update({'|'.join(record.id.split('|')[0:2]): record}) # append record list under genome_tag

In [21]:
# for each PCG
for mag_gff_file in pcg_gffs:
    # get MAG tag
    mag_tag = mag_gff_file.split('/')[3]
    # save to file
    output_faa_file = '../results/ME_PATRIC_annotated_features/ME.{0}.PATRIC_annotated_features.faa'.format(mag_tag)
    if not os.path.exists(output_faa_file):
        # charge annotation file
        mag_gff = pd.read_csv(mag_gff_file, 
                              sep = '\t', 
                              comment = '#',
                              names=['Contig', 'Annotation source', 'Feature type', 'Start', 'End', 'extra', 'Strand', 'E-value', 'ID'])
        mag_gff['Contig'] = [contig.replace('accn|', '') for contig in mag_gff['Contig'].to_list()] # modify contig name
        mag_gff
        # get mag target contig
        mag_ME_fasta_file = '../data/mobile_elements_sequences/ME.{0}.fna'.format(mag_tag)
        mag_target_contig = pcg_blast_dict[mag_tag]['sseqid'].to_list()[0]
        mag_target_start = pcg_blast_dict[mag_tag]['sstart'].to_list()[0]
        mag_target_end = pcg_blast_dict[mag_tag]['send'].to_list()[0]
        # subset for target contig
        mag_target_contig
        # get features of the contig
        mag_ME_annotated_proteins_table = mag_gff.query("`Contig` in @mag_target_contig & `Feature type` == 'CDS' & Start >= @mag_target_start & End <= @mag_target_end")
        mag_ME_annotated_proteins_IDs = [feature.split(';')[0].replace('ID=', '') for feature in mag_ME_annotated_proteins_table['ID'].to_list()]
        
        # subsetting proteins
        mag_ME_protein_seqs = [pcg_protein_dict[mag_tag][protein_id] for protein_id in mag_ME_annotated_proteins_IDs if protein_id in pcg_protein_dict[mag_tag].keys()]
        
        # modifying ID and description in order to remove white spaces (disturbs future analyses)
        #for record in mag_ME_protein_seqs:
        #    record.id = record.id.replace(' ', '_')
        #    record.description = record.description.replace(' ', '_')
        
        # checking that number of extracted sequences is OK
        if mag_ME_annotated_proteins_table.shape[0] != len(mag_ME_protein_seqs):
            print('Problem extracting sequences of MAG:', mag_tag)
        elif mag_ME_annotated_proteins_table.shape[0] == len(mag_ME_protein_seqs):
            print('Sequence extraction went OK for MAG:', mag_tag)
        # save to file
        output_faa_file = '../results/ME_PATRIC_annotated_features/ME.{0}.PATRIC_annotated_features.faa'.format(mag_tag)
        if not os.path.exists(output_faa_file):
            with open(output_faa_file, 'w') as output_handle:
                SeqIO.write(mag_ME_protein_seqs, output_handle, "fasta")

In [23]:
# list all MAG contigs
halovivax_gff_files = glob.glob('../data/genome_annotations/017357405.1/017357405.1.gff')

# for each MAG
for mag_gff_file in halovivax_gff_files:
    # get MAG tag
    mag_tag = mag_gff_file.split('/')[3]
    
    # charge annotation file
    mag_gff = pd.read_csv(mag_gff_file, 
                          sep = '\t', 
                          comment = '#',
                          names=['Contig', 'Annotation source', 'Feature type', 'Start', 'End', 'extra', 'Strand', 'E-value', 'ID'])
    mag_gff
    # get mag target contig
    mag_ME_fasta_file = '../data/mobile_elements_sequences/ME.{0}.fna'.format(mag_tag)
    mag_target_contig = halovivax_blast_hit['sseqid'].to_list()
    mag_target_start = halovivax_blast_hit['sstart'].to_list()[0]
    mag_target_end = halovivax_blast_hit['send'].to_list()[0]
    # subset for target contig
    mag_target_contig
    # get features of the contig
    mag_ME_annotated_proteins_table = mag_gff.query("`Contig` in @mag_target_contig & `Feature type` == 'CDS' & Start >= @mag_target_start & End <= @mag_target_end")
    mag_ME_annotated_proteins_IDs = [feature.split(';')[0].replace('ID=cds-', '') for feature in mag_ME_annotated_proteins_table['ID'].to_list()]
    
    # subsetting proteins
    mag_ME_protein_seqs = [halovivax_proteins_dict[protein_id] for protein_id in mag_ME_annotated_proteins_IDs]
    
    # modifying ID and description in order to remove white spaces (disturbs future analyses)
    for record in mag_ME_protein_seqs:
        record.id = record.id.replace(' ', '_')
        record.description = record.description.replace(' ', '_')
    
    # checking that number of extracted sequences is OK
    if mag_ME_annotated_proteins_table.shape[0] != len(mag_ME_protein_seqs):
        print('Problem extracting sequences of MAG:', mag_tag)
    elif mag_ME_annotated_proteins_table.shape[0] == len(mag_ME_protein_seqs):
        print('Sequence extraction went OK for MAG:', mag_tag)
    # save to file
    output_faa_file = '../results/ME_PATRIC_annotated_features/ME.{0}.PATRIC_annotated_features.faa'.format(mag_tag)
    if not os.path.exists(output_faa_file):
        with open(output_faa_file, 'w') as output_handle:
            SeqIO.write(mag_ME_protein_seqs, output_handle, "fasta")

Sequence extraction went OK for MAG: 017357405.1


# Infer groups of homologous sequences 
Groups of homologous sequences are infered with *get_homologues*.

In [29]:
%%bash
#nohup /home/mauri/programas/get_homologues-x86_64-20210305/get_homologues.pl -e -d ../results/MEs_predicted_orfs/ -t 0 -m local -n 10 -C 70 -S 35 -M 1 &> nohup.get_homologues_ORFs_identity35_no_inparalogs.out &
#mv MEs_predicted_orfs_homologues ../results

# Stablishing a correlation between predicted ORFs and annotated features
Also renaming ORFs in order to get nice names to work with. Saving a correlation table afterwards. Also subsetting homolog groups containing at least one annotated feature in respective source database.

Running in a separate notebook (parsing_orf_homolog_groups.ipynb) because of conda environment incompatibilities.

## Copying genome files of MAGs and metagenomes into common folder with those of PCG

In [37]:
# create auxiliary function
def create_dir(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)

# copying process for MAGs
# listing MAGs genome files
genome_fna_files = [fna_file for fna_file in glob.glob('../data/MAGS/ftp.ncbi.nlm.nih.gov/*/*/*/*/*/*/*/*v*_genomic.fna') if not re.match(pattern = '.*rna_from.*|.*cds_from.*', string = fna_file)]
# iterate over files
for fna_file in genome_fna_files:
    # get MAG tag
    mag_tag = fna_file.split('/')[10].split('_')[1]
    # create allocating folder
    output_mag_genome_folder = '../data/genome_annotations/{0}'.format(mag_tag)
    create_dir(output_mag_genome_folder)
    # create output file name
    output_mag_genome_file = '{0}/{1}.fna'.format(output_mag_genome_folder, mag_tag)
    # copy from source location to destination folder
    shutil.copyfile(src = fna_file, dst = output_mag_genome_file)
    
# copying process for metagenomes
# listing metagenome files
metagenome_fna_files = glob.glob('../data/mobile_elements_sequences_metagenomes/*fasta')
# iterating over files
for metagenome_fna_file in metagenome_fna_files:
    # get metagenome tag
    metagenome_tag = metagenome_fna_file.split('/')[3].replace('.fasta', '').lower().split('.')[0]
    # create allocating folder
    output_metagenome_genome_folder = '../data/genome_annotations/{0}'.format(metagenome_tag)
    create_dir(output_metagenome_genome_folder)
    # create output file name
    output_metagenome_genome_file = '{0}/{1}.fna'.format(output_metagenome_genome_folder, metagenome_tag)
    # copy from source location to destination folder
    shutil.copyfile(src = metagenome_fna_file, dst = output_metagenome_genome_file)

## Updating *genomes_and_taxonomy.csv* with info of MAGs and metagenomes
Species names and their genome ID codes are stored in the table *genomes_and_taxonomy.csv*, which already has information regarding PCG Archaea.
API requests are performed to get information for MAGs (from **GTDB database**; https://gtdb.ecogenomic.org/) and metagenomes (from **JGI portal**; https://genome.jgi.doe.gov/).

In [38]:
# import library to perform API calls
import requests

# trying first API request for GTDB
# create list to allocate pandas DataFrame with data
mag_species_data = []

# getting MAG Genome IDs
mag_genome_ids = [mag_gff_file.split('/')[10].rpartition('_')[0] for mag_gff_file in mag_gff_files]
# adding mag_genome_id for MAG without annotation
mag_genome_ids = mag_genome_ids + ['GCA_001563915.1']
# iterating over MAG IDs
for mag_genome_id in mag_genome_ids:
    # performing API request
    response = requests.get('https://gtdb.ecogenomic.org/api/v1/genome/summary/{0}'.format(mag_genome_id))
    # getting request JSON data structure
    mag_genome_json = response.json()
    # getting species name for Genome ID
    # contamplating possible scenarios
    if mag_genome_json['gtdb_species'] != 's__':
        mag_genome_phylum = mag_genome_json['gtdb_phylum'].split('p__')[1].replace(' ', '_')
        mag_genome_species = mag_genome_json['gtdb_species'].split('s__')[1].replace(' ', '_')        
    elif mag_genome_json['gtdb_species'] == 's__':
        mag_genome_phylum = mag_genome_json['ncbi_taxonomy'].split(';')[1].split('p__')[1].replace(' ', '_')
        mag_genome_species = mag_genome_json['ncbi_taxonomy'].split(';')[6].split('s__')[1].replace(' ', '_')
    # contamplating empty string for species
    if mag_genome_species == '':
        mag_genome_species = mag_genome_id.rpartition('.')[0]
    # adding pandas DataFrame to <mag_species_data>
    mag_genome_full_data = '_'.join([mag_genome_phylum, mag_genome_species])
    mag_species_data.append(pd.DataFrame.from_dict({'Genome_id': [mag_genome_id], 'Taxonomy': [mag_genome_full_data]}))
# concatenating to create table
mag_species_data_table = pd.concat(mag_species_data)

# the case of metagenomes is a bit different because metadata seems to have long and uninformative/too-informative names, with no taxon clasiffication for contigs
# going to perform a replication of genome id in this case
metagenome_species_data = []
# listing metagenome files
metagenome_fna_files = glob.glob('../data/mobile_elements_sequences_metagenomes/*fasta')
# iterating over files
for metagenome_fna_file in metagenome_fna_files:
    # get metagenome tag
    metagenome_tag = metagenome_fna_file.split('/')[3].replace('.fasta', '').lower().split('.')[0]
    # appending pandas DataFrame
    metagenome_species_data.append(pd.DataFrame.from_dict({'Genome_id': [metagenome_tag], 'Taxonomy': [metagenome_tag]}))
metagenome_species_data_table = pd.concat(metagenome_species_data)

In [39]:
# opening old file and saving new
genome2taxonomy_table = pd.read_csv('../data/genomes_and_taxonomy.csv', sep = ',')
# create row for Halovivax
halovivax_data_table = pd.DataFrame.from_dict({'Genome_id': ['017357405.1'], 'Taxonomy': ['Halovivax_sp_KZCA124']})
# concatenating tables
genome2taxonomy_table_full = [genome2taxonomy_table, halovivax_data_table, mag_species_data_table, metagenome_species_data_table]
genome2taxonomy_full = pd.concat(genome2taxonomy_table_full)
# getting rid of duplicates (in case of re-running script)
genome2taxonomy_full = genome2taxonomy_full.drop_duplicates()
# saving new table
genome2taxonomy_full.loc[:, ['Genome_id', 'Taxonomy']].to_csv('../data/genomes_and_taxonomy.csv', index = False)

In [40]:
pd.DataFrame.drop_duplicates(genome2taxonomy_full.loc[:, ['Genome_id', 'Taxonomy']])

,Genome_id,Taxonomy
0,1227494.3,Natrinema_altunense_JCM_12890
1,1526048.3,Haloferax_sp_Q22
2,222984.5,Natrinema_altunense_strain_AJ2
3,2496101.3,Haloterrigena_sp_SYSU_A121-1
4,2743089.3,Halobonum_sp_NJ-3-1
5,60847.21,Halogeometricum_borinquense_strain_wsp4
6,926690.3,Haloplanus_natans_DSM_17983
7,GCA_011039335.1,Proteobacteria_Candidatus_Desulfofervidus_auxilii
8,GCA_011380245.1,Thermoproteota_DSZF01_sp011372955
9,GCA_011358415.1,Micrarchaeota_DTNV01_sp011358415


In [41]:
%load_ext rpy2.ipython

In [42]:
# creating directories to allocate
target_dirs = ['../results/MEs_predicted_orfs_renamed', 
               '../results/MEs_predicted_orfs_renamed/results', 
               '../results/MEs_predicted_orfs_renamed/data']

[create_dir(dir) for dir in target_dirs]

[None, None, None]

In [43]:
%%R -o orfs_annotation_table

# loading libraries
library(tidyverse)
library(magrittr)
library(glue)
library(bioseq)
library(tidytidbits)

# loading table relating species and genome ID
genomes_and_taxonomy.tibble = readr::read_csv('../data/genomes_and_taxonomy.csv', col_names = T) %>% dplyr::mutate(Genome_id = Genome_id %>% as.character())
genomes_and_taxonomy.dict = genomes_and_taxonomy.tibble$Taxonomy
names(genomes_and_taxonomy.dict) = genomes_and_taxonomy.tibble$Genome_id %>% str_replace_all(., 'GCA_', '')

# loading genomes and creating a dictionary for contig names and contig code
contig_codes_and_labels.tibble = tibble(genome_fasta = list.files('../data/genome_annotations', pattern = '.fna$', full.names = T, recursive = T)) %>%
  group_split(genome_fasta) %>%
  purrr::map_dfr(., ~{fasta = .x$genome_fasta
                  fasta_vct = bioseq::read_fasta(fasta, type = 'DNA') 
                      
                  fasta_vct %>% 
                      as_tibble() %>%
                      dplyr::rename(sequence = 'value') %>%
                      dplyr::mutate(label = names(fasta_vct)) %>%
                      dplyr::mutate(contig_code = label %>% str_split(' ') %>% purrr::map_chr(1))
                  })

code2contig.dict = contig_codes_and_labels.tibble$label
names(code2contig.dict) = contig_codes_and_labels.tibble$contig_code

# loading correspondence between ORFs and annotated features
RBH.tibble = readr::read_tsv('../results/MEs_annotated_features_vs_predicted_orfs_BRHs.tsv', col_names = T)
annotated2ORF.dict = RBH.tibble$query_id
names(annotated2ORF.dict) = RBH.tibble$subject_id

# loading ORFs FASTAs
orfs_annotation.tibble = tibble(FASTA_file = list.files('../results/MEs_predicted_orfs', pattern = '.faa', full.names = T)) %>%
  # stablishing organism code
  dplyr::mutate(genome_id = FASTA_file %>% str_split('/') %>% purrr::map_chr(4) %>% str_replace_all(., 'ME.|.predicted_orfs.faa|.MAG|.metagenome', ''),
                species_code = genome_id %>% tidytidbits::lookup_chr(., dict = genomes_and_taxonomy.dict)) %>%
  # loading FASTAs
  group_split(FASTA_file) %>%
  purrr::map_dfr(., ~{
                 file = .x$FASTA_file
                 genome_id = .x$genome_id
                 species_code = .x$species_code
    
                 fasta_vct = bioseq::read_fasta(file, type = 'AA') 
                 
                 fasta_vct %>%
                 as_tibble() %>%
                 dplyr::mutate(label = names(fasta_vct)) %>%
                 dplyr::rename(sequence = 'value') %>%
                 # renaming sequences and parsing data related to strandedness, origin and stop
                 dplyr::mutate(genome_id = genome_id, 
                               new_label = glue('ORF_{1:nrow(.)}_{species_code}'),
                               annotated_feature_name = label %>% 
                                  str_split(' ') %>% 
                                  purrr::map_chr(1) %>% 
                                  tidytidbits::lookup_chr(., dict = annotated2ORF.dict),
                               contig = label %>% str_split(' ') %>% 
                                  purrr::map_chr(1) %>% 
                                  str_split('_') %>%
                                  purrr::map_chr(., ~{.[-length(.)] %>% paste(collapse = '_')}) %>%
                                  tidytidbits::lookup_chr(., dict = code2contig.dict, default = identity),
                               strand = case_when(str_detect(label, 'REVERSE') ~ '-', TRUE ~ '+'),
                               START = label %>% str_split(' ') %>% purrr::map_chr(2) %>% str_replace_all(., '\\[', '') %>% as.double(),
                               STOP = label %>% str_split(' ') %>% purrr::map_chr(4) %>% str_replace_all(., '\\]', '') %>% as.double())
                 }) %>%
  dplyr::rename(sequence_AA = 'sequence', `Genome ID` = 'genome_id', `ORF ID` = 'new_label', `Featured ID in Genome Annotation` = 'annotated_feature_name',
               `getorf featured ID` = 'label') %>%
  dplyr::relocate(., `Genome ID`, `ORF ID`, `Featured ID in Genome Annotation` , `getorf featured ID`, contig, strand, START, STOP, sequence_AA)
  
# saving table
orfs_annotation.tibble %>% dplyr::select(-sequence_AA) %>% readr::write_tsv('../results/MEs_predicted_orfs_renamed/data/annotation_table_ORFs_renamed.tsv', col_names = T)

# create table to examine annotation in python
orfs_annotation_table = orfs_annotation.tibble 

R[write to console]: ── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: 
Attaching package: ‘magrittr’


R[write to console]: The following object is masked from ‘package:purrr’:

    set_names


R[write to console]: The following object is masked from ‘package:tidyr’:

    extract


R[write to console]: 
Attaching package: ‘glue’


R[write to console]: The following object is masked from ‘package:dplyr’:

    collapse





── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  Genome_id = col_character(),
  Taxonomy = col_character()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  query_id = col_character(),
  subject_id = col_character()
)
ℹ Use `spec()` for the full column specifications.



# Create MSA for homolog groups 
Running MAFFT under L-INS-I method for this purpose.

In [45]:
# script made to align clusters of homologous sequences with MAFFT running under L-INS-I algorithm
# import libraries
import os
import subprocess
import glob

# creating directory to allocate alignments
if not os.path.exists('../results/MEs_predicted_orfs_homolog_alns'):
  os.mkdir('../results/MEs_predicted_orfs_homolog_alns')

# listing homologs groups
homolog_groups_data = [(file.replace('.faa', '.msa'),
                        file) for file in os.listdir('../results/MEs_predicted_orfs_filtered_homologous_groups') if file.endswith('.faa')]

# running alignments
for set in homolog_groups_data:
  # unpacking variables
  msa_output, homolog_cluster_file = set
  if not os.path.exists('../results/MEs_predicted_orfs_homolog_alns/{0}'.format(msa_output)):
    out_file = open('../results/MEs_predicted_orfs_homolog_alns/{0}'.format(msa_output), 'w') 
    mafft_command = 'mafft --maxiterate 1000 --localpair ../results/MEs_predicted_orfs_filtered_homologous_groups/{0}'.format(homolog_cluster_file).split(' ')
    subprocess.run(mafft_command, stdout = out_file)


# Perform remote homologs searches for each homolog group

## Perform search with *jackhmmer* against UniRef50

In [48]:
# script made to extend groups of homologues already found with distant homolog searches
# importing libaries
import os
import subprocess
import glob
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from io import StringIO
from collections import defaultdict
from Bio import SearchIO
import urllib

# loading table of homologous groups founded, and getting those with more than two sequences
homolog_groups_with_ORFs_table = pd.read_csv('../results/homolog_groups_tables/homolog_groups_with_ORFs.tsv', sep = '\t')

# creating directory to allocate files created in this script
dirs_to_be_created = ['../results/homolog_groups_expansion_and_collapse/results/jackhmmer_hits_fastas',
                      '../results/homolog_groups_expansion_and_collapse/results/jackhmmerouts',
                      '../results/homolog_groups_expansion_and_collapse/results/jackhmmeralns',
                      '../results/homolog_groups_expansion_and_collapse/results/jackhmmertblouts',
                      '../results/homolog_groups_expansion_and_collapse/results/jackhmmerdomtblouts',
                      '../results/homolog_groups_expansion_and_collapse/results/jackhmmerhmms',
                      '../results/homolog_groups_expansion_and_collapse/results/jackhmmer_hits_alns',
                      '../results/homolog_groups_expansion_and_collapse/results/homolog_groups_HMMs',
                      '../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results',
                      '../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results/hmmsearchouts',
                      '../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results/tblouts',
                      '../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results/domtblouts']

for dir in dirs_to_be_created:
  if not os.path.exists(dir):
    os.mkdir(dir)
    
# running jackhmmer with fasta files
for fasta_file in homolog_groups_with_ORFs_table.File.to_list():
  if os.path.exists(fasta_file):
    # getting tag of group
    tag = fasta_file.split('/')[3].replace('.faa', '')
    database = '../../../../databases/uniref50.fasta'
    # running with 1 iterations
    if not os.path.exists('../results/homolog_groups_expansion_and_collapse/results/jackhmmerouts/{0}.jackhmmer.out'.format(tag)):
      command = 'jackhmmer -N 1 -o ../results/homolog_groups_expansion_and_collapse/results/jackhmmerouts/{0}.jackhmmer.out -A ../results/homolog_groups_expansion_and_collapse/results/jackhmmeralns/{0}.jackhmmer.aln --tblout ../results/homolog_groups_expansion_and_collapse/results/jackhmmertblouts/{0}.tblout --domtblout ../results/homolog_groups_expansion_and_collapse/results/jackhmmerdomtblouts/{0}.domblout --chkhmm ../results/homolog_groups_expansion_and_collapse/results/jackhmmerhmms/{0}.hmms --cpu 15 {1} {2}'.format(tag, fasta_file, database).split(' ')
      # run jackhmmer
      subprocess.run(command)

../results/MEs_predicted_orfs_filtered_homologous_groups/43214_NZ_CP071597.1_1987.faa
../results/MEs_predicted_orfs_filtered_homologous_groups/41684_NZ_CP071597.1_457.faa
../results/MEs_predicted_orfs_filtered_homologous_groups/33690_QNAN01000020.1_20.faa
../results/MEs_predicted_orfs_filtered_homologous_groups/41631_NZ_CP071597.1_404.faa
../results/MEs_predicted_orfs_filtered_homologous_groups/43868_NZ_CP071597.1_2641.faa
../results/MEs_predicted_orfs_filtered_homologous_groups/43865_NZ_CP071597.1_2638.faa
../results/MEs_predicted_orfs_filtered_homologous_groups/43833_NZ_CP071597.1_2606.faa
../results/MEs_predicted_orfs_filtered_homologous_groups/47161_JNCS01000001_1324.faa
../results/MEs_predicted_orfs_filtered_homologous_groups/43797_NZ_CP071597.1_2570.faa
../results/MEs_predicted_orfs_filtered_homologous_groups/41611_NZ_CP071597.1_384.faa
../results/MEs_predicted_orfs_filtered_homologous_groups/44824_LOEP01000012_734.faa
../results/MEs_predicted_orfs_filtered_homologous_groups/4277

## Tabulation of *jackhmmer* results

In [50]:
%%R 
# script made to tabulate some domtblouts tables generated with jackhmmer3
# loading libraries
library(rhmmer)
library(tidyverse)
library(glue)
library(magrittr)

# define column names, as the package assumes headers different from what is actually written in the files
domtblouts_colnames = c('target_name', 't_accession', 'tlen', 'query_name', 'q_accession', 'qlen',
                    'fullseq_evalue', 'fullseq_score', 'fullseq_bias', 'num_of_domain', 'total_hit_domains', 
                    'c-evalue', 'i-evalue', 'hmm_score', 'hmm_bias', 'hmm_coord_from', 'hmm_coord_to', 'ali_coord_from',
                    'ali_coord_to', 'env_coord_from', 'env_coord_to', 'acc', 'description_of_target')

# listing domtblouts files
domtblout_files = list.files('../results/homolog_groups_expansion_and_collapse/results/jackhmmerdomtblouts', pattern = '.domblout', full.names = T)

# iterating over files and creating output files
for (i in seq_along(domtblout_files)) {
  # reading file
  current_domtblout.tibble = rhmmer::read_domtblout(file = domtblout_files[i])
  
  # changing column names
  colnames(current_domtblout.tibble) = domtblouts_colnames
  
  # getting file tag
  group_tag = domtblout_files[i] %>% str_split('/') %>% purrr::map_chr(6)
  
  # filtering based on target and query lengths, evalues and query coverage of alignment
  # then exporting TSV
  
  # export the original table
  current_domtblout.tibble %>% readr::write_tsv(glue('../results/homolog_groups_expansion_and_collapse/results/jackhmmerdomtblouts_tsvs/{group_tag}.tsv'), col_names = T)
  
  # export filtered tables
  current_domtblout.tibble %>%
    # target must have at least 300aa and be no more than 1.5 times the query length
    dplyr::filter((100 < tlen) & (tlen <= qlen*1.5)) %>%
    rowwise() %>%
    # creating column with alignment length and getting qcov
    dplyr::mutate(aln_length = abs(ali_coord_to - ali_coord_from),
                  qcov = aln_length/qlen) %>%
    # query coverage must be at least 50%
    dplyr::filter(qcov >= 0.5) %>%
    # global sequence e-value must be at most 1e-15
    dplyr::filter(fullseq_evalue <= 1e-15) %>%
    # saving table
    readr::write_tsv(glue('../results/homolog_groups_expansion_and_collapse/results/jackhmmerdomtblouts_tsvs/{group_tag}.filtered.tsv'), col_names = T)
}

|====================================================================| 100% 7 MB


## Following with remote homology inference (hmmsearch stuff)

In [51]:
# script made to extend groups of homologues already found with distant homolog searches
# importing libaries
import os
import subprocess
import glob
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from io import StringIO
from collections import defaultdict
from Bio import SearchIO
import urllib

# loading table of homologous groups founded, and getting those with more than two sequences
homolog_groups_with_ORFs_table = pd.read_csv('../results/homolog_groups_tables/homolog_groups_with_ORFs.tsv', sep = '\t')

# creating directory to allocate files created in this script
dirs_to_be_created = ['../results/homolog_groups_expansion_and_collapse/results/jackhmmer_hits_fastas',
                      '../results/homolog_groups_expansion_and_collapse/results/jackhmmer_hits_alns',
                      '../results/homolog_groups_expansion_and_collapse/results/homolog_groups_HMMs',
                      '../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results',
                      '../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results/hmmsearchouts',
                      '../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results/tblouts',
                      '../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results/domtblouts']

for dir in dirs_to_be_created:
  if not os.path.exists(dir):
    os.mkdir(dir)
    
# getting ORF sequences
orfs_set = [(orfs_file.split('/')[3].replace('ME.', '').replace('.fna', ''),
               orfs_file) for orfs_file in glob.glob('../results/MEs_predicted_orfs/ME.*')]

# loading ORFs with SeqIO.parse
orfs_dict = {}
for data_set in orfs_set:
  # unpacking variables
  genome_tag, orfs_file = data_set
  genome = SeqIO.parse(orfs_file, 'fasta')
  # creating dictionary with sequences, descriptions and genome code
  genome_dict = {}
  for seq in genome:
    orfs_dict.update({seq.name: seq})

# saving ORFs in FASTA
if not os.path.exists('../results/homolog_groups_expansion_and_collapse/data/all_predicted_orfs.faa'):
  with open('../results/homolog_groups_expansion_and_collapse/data/all_predicted_orfs.faa', "w") as output_handle:
      SeqIO.write(list(orfs_dict.values()), output_handle, "fasta")
    
# iterating over files of domtblouts of jackhmmer and performing
list_domtblouts_filtered = glob.glob('../results/homolog_groups_expansion_and_collapse/results/jackhmmerdomtblouts_tsvs/*filtered.tsv')

# getting all hits
all_uniref50_hits = []
for domtblout_file in list_domtblouts_filtered:
  # reading table
  domtblout_table = pd.read_csv(domtblout_file, sep = '\t', comment = '#')
  # getting hits
  hits = domtblout_table.target_name.to_list()
  # appending hits
  for hit in hits:
    all_uniref50_hits.append(hit)

#all_uniref50_hits = list(set(all_uniref50_hits))
all_uniref50_hits = list(dict.fromkeys(all_uniref50_hits))
# saving FASTA file
output_fasta_uniref50filtered = '../results/homolog_groups_expansion_and_collapse/data/uniref50_hitted_entries.faa'
if not os.path.exists(output_fasta_uniref50filtered):
  # subsetting UniRef50 FASTA in order to create a FASTA containing only UniRef hits (in order to subset it in the future)
  uniref50_fasta = SeqIO.parse('../../../../databases/uniref50.fasta', 'fasta')
  
  uniref50_hitted = {}
  while len(uniref50_hitted) < len(all_uniref50_hits):
    for record in uniref50_fasta:
      if record.id in all_uniref50_hits:
        print('adding', record.id)
        uniref50_hitted.update({record.id: record})

  with open(output_fasta_uniref50filtered, "w") as output_handle:
    SeqIO.write(list(uniref50_hitted.values()), output_handle, "fasta")

if os.path.exists(output_fasta_uniref50filtered):
  # load fasta
  uniref50_hitted_fasta = SeqIO.parse(output_fasta_uniref50filtered, 'fasta')
  # create dictionary uniref50_hitted
  uniref50_hitted = {}
  [uniref50_hitted.update({record.id: record}) for record in uniref50_hitted_fasta]
    
for filename in list_domtblouts_filtered:
  # to create the script, an example # filename = '../results/homolog_groups_expansion_and_collapse/results/jackhmmerdomtblouts_tsvs/10721_CP048739_181.domblout.filtered.tsv'
  #group_tag = filename.split('/')[5].split('.')[0]
  group_tag = filename.split('/')[5].replace('.domblout.filtered.tsv', '')

  colnames_domblout = ['target_name', 't_accession', 'tlen', 'query_name', 'q_accession', 'qlen',
                      'fullseq_evalue', 'fullseq_score', 'fullseq_bias', 'num_of_domain', 'total_hit_domains', 
                      'c-evalue', 'i-evalue', 'hmm_score', 'hmm_bias', 'hmm_coord_from', 'hmm_coord_to', 'ali_coord_from',
                      'ali_coord_to', 'env_coord_from', 'env_coord_to', 'acc', 'description_of_target', 'aln_length', 'qcov']
  
  group_jackhmmer_domtable = pd.read_csv(filename, 
                                         sep = '\t', 
                                         comment = '#')
  
  # getting sequences from UniRef50 and from ORF files
  set_of_seqs_dict = {}
  
  # getting sequences from ORFs in the group
  #orfs_in_group = set(group_jackhmmer_domtable.query_name.to_list())
  # loading fasta of group
  group_fasta = SeqIO.parse('../results/MEs_predicted_orfs_filtered_homologous_groups/{0}.faa'.format(group_tag), 'fasta')
  orfs_in_group = [record.id for record in group_fasta]
  for orf_id in orfs_in_group:
    # retrieve from orf dictionary and appending record
    set_of_seqs_dict.update({orf_id: orfs_dict[orf_id]})
      
  # creating set variable with IDs of hits from UniRef50
  #uniref50_hits = set(group_jackhmmer_domtable.target_name.to_list())
  uniref50_hits = list(dict.fromkeys((group_jackhmmer_domtable.target_name.to_list())))
  
  # retrieving with the UniRef API
  for uniref_id in uniref50_hits:
    # retrieving sequences from uniref50_hitted and appending to set_of_seqs_dict
    [set_of_seqs_dict.update({record.id: record}) for record in [uniref50_hitted[uniref_id]]]
    
  # saving group FASTA into a file
  output_fasta_file = '../results/homolog_groups_expansion_and_collapse/results/jackhmmer_hits_fastas/{0}.group_members_and_jackhmmerhits.faa'.format(group_tag)
  if not os.path.exists(output_fasta_file):
    with open(output_fasta_file, "w") as output_handle:
      SeqIO.write(list(set_of_seqs_dict.values()), output_handle, "fasta")
      
  # aligning sequences based on HMM profile
  hit_aln = '../results/homolog_groups_expansion_and_collapse/results/jackhmmer_hits_alns/{0}.msa'.format(group_tag)
  #hmm_group_jackhmmer = '../results/homolog_groups_expansion_and_collapse/results/jackhmmerhmms/{0}.hmms-1.hmm'.format(group_tag)
  if not os.path.exists(hit_aln):
    # running hmmalign
    #hmmalign_cmd = 'hmmalign -o {0} {1} {2}'.format(hit_aln, hmm_group_jackhmmer, output_fasta_file).split(' ')
    #subprocess.run(hmmalign_cmd)
    # running MAFFT
    out_file = open(hit_aln, 'w') 
    mafft_command = 'mafft --auto {0}'.format(output_fasta_file).split(' ')
    subprocess.run(mafft_command, stdout = out_file)
    
  # creating new profile
  homolog_group_hmmout = '../results/homolog_groups_expansion_and_collapse/results/homolog_groups_HMMs/{0}.hmm'.format(group_tag)
  if not os.path.exists(homolog_group_hmmout):
    hmmbuild_cmd = 'hmmbuild --cpu 3 {0} {1}'.format(homolog_group_hmmout, hit_aln).split(' ')
    subprocess.run(hmmbuild_cmd)
  
  # searching against ORFs with hmmsearch
  hmmsearchout = '../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results/hmmsearchouts/{0}.hmmsearchout'.format(group_tag)
  hmmsearchtblout = '../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results/tblouts/{0}.tblout'.format(group_tag)
  hmmsearchdomtblout = '../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results/domtblouts/{0}.domtblout'.format(group_tag)
  if not os.path.exists(hmmsearchout):
    hmmsearch_cmd = 'hmmsearch -o {0} --tblout {1} --domtblout {2} --cpu 3 {3} ../results/homolog_groups_expansion_and_collapse/data/all_predicted_orfs.faa'.format(hmmsearchout, hmmsearchtblout, hmmsearchdomtblout, homolog_group_hmmout, ).split(' ')
    subprocess.run(hmmsearch_cmd)

## Parsing of tabulated *hmmsearch*'s results table

In [54]:
%%R 
# script made to parse hmmsearch results of homologous groups against ORFs
# loading libraries
library(rhmmer)
library(tidyverse)
library(magrittr)
library(glue)

# create vector with column names of hmmsearch output
domtblouts_colnames = c('target_name', 't_accession', 'tlen', 'query_name', 'q_accession', 'qlen',
                    'fullseq_evalue', 'fullseq_score', 'fullseq_bias', 'num_of_domain', 'total_hit_domains', 
                    'c-evalue', 'i-evalue', 'hmm_score', 'hmm_bias', 'hmm_coord_from', 'hmm_coord_to', 'ali_coord_from',
                    'ali_coord_to', 'env_coord_from', 'env_coord_to', 'acc', 'description_of_target')

# see how many hits has each homologous group
filtered_hits.tibble = list.files('../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results/domtblouts', pattern = '.domtblout', full.names=T) %>%
  as.list() %>%
  purrr::map_dfr(., ~{
    # load results
    current_domtblout.tibble = rhmmer::read_domtblout(file = .x)
    # rename columns
    colnames(current_domtblout.tibble) = domtblouts_colnames
    # filtering results
    current_domtblout.tibble %>%
      # target must have at least 300aa and be no more than 1.5 times the query length
      #dplyr::filter((0.5*qlen < tlen) & (tlen <= qlen*1.5)) %>%
      rowwise() %>%
      # creating column with alignment length and getting qcov
      dplyr::mutate(aln_length = abs(ali_coord_to - ali_coord_from),
                    qcov = aln_length/qlen) %>%
      # query coverage must be at least 50%
      dplyr::filter(qcov >= 0.5) %>%
      # global sequence e-value must be at most 1e-15
      dplyr::filter(fullseq_evalue <= 1e-10)
                 })

# saving table
filtered_hits.tibble %>% readr::write_tsv(., '../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results/hmmsearches_filtered_per_group.tsv')

# getting number of hits and number of organisms hitted by each family and saving results
filtered_hits.tibble %>% 
  ungroup() %>% 
  group_split(query_name) %>%
  purrr::map_dfr(., ~{
      hits = unique(.x$target_name)
      group_tag = unique(.x$query_name)
      organisms = .x$target_name %>% str_split('_') %>% purrr::map_chr(1)
      organisms = unique(organisms)
      tibble(homolog_group = group_tag, number_of_hits = length(hits), number_of_organisms = length(organisms))
  }) %>%
dplyr::arrange(desc(number_of_hits)) %>%
readr::write_tsv(., '../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results/hmmsearches_number_of_hits_per_group.tsv')

filtered_hits.tibble

# A tibble: 1,404 x 25
# Rowwise: 
   target_name   t_accession  tlen query_name   q_accession  qlen fullseq_evalue
   <chr>         <chr>       <int> <chr>        <chr>       <int>          <dbl>
 1 CP010426.1_1… -              53 10196_CP010… -              53       1.3e- 37
 2 CP058579_203  -              47 10196_CP010… -              53       1.9e- 29
 3 CP010426.1_1… -             111 11192_CP010… -             123       1.6e- 42
 4 LKMP01000007… -             117 11192_CP010… -             123       8  e- 29
 5 Ga0207715_10… -             421 11406_CP010… -             394       2.1e-139
 6 CP010426.1_1… -             398 11406_CP010… -             394       3.8e-136
 7 VEPM01000029… -             122 12736_CP010… -             112       1.2e- 41
 8 CP010426.1_1… -              98 12736_CP010… -             112       4.9e- 33
 9 CP010426.1_1… -             174 16193_CP010… -             208       1.3e- 49
10 Ga0172379_10… -             208 16193_CP010… -             208       1.

## Perform pairwise searches between groups and collapse them (hmmalign)

In [55]:
# load libraries
import os
import glob
import subprocess
import itertools
import networkx as nx
from Bio.SearchIO import HHsuiteIO
from Bio import SeqIO, AlignIO
from Bio.SeqIO import SeqRecord
import pandas as pd
import re
import pyhmmer

# list HMM files
hmm_list = [(hmm.split('/')[5].replace('.hmm', ''), hmm) for hmm in glob.glob('../results/homolog_groups_expansion_and_collapse/results/homolog_groups_HMMs/*hmm')]

# create directory to allocate results
important_dirs = ['../results/homolog_groups_expansion_and_collapse/results/homolog_groups_HHMs', 
                  '../results/homolog_groups_expansion_and_collapse/results/hhalign', 
                  '../results/homolog_groups_expansion_and_collapse/results/hhalign/outfiles',
                  '../results/homolog_groups_expansion_and_collapse/results/hhalign/tsv',
                  '../results/homolog_groups_expansion_and_collapse/results/connected_groups',
                  '../results/homolog_groups_expansion_and_collapse/results/connected_groups/fastas',
                  '../results/homolog_groups_expansion_and_collapse/results/connected_groups/fastas/protein',
                  '../results/homolog_groups_expansion_and_collapse/results/connected_groups/fastas/CDS',
                  '../results/homolog_groups_expansion_and_collapse/results/connected_groups/tsv']
for dir in important_dirs:
  if not os.path.exists(dir):
    os.mkdir(dir)

# create HHM files with hhmake
for set in hmm_list:
  # depack variables
  group_tag, hmm_file = set
  # create hhm file
  hhm_file = '../results/homolog_groups_expansion_and_collapse/results/homolog_groups_HHMs/{0}.hhm'.format(group_tag)
  if not os.path.exists(hhm_file):
    hhmake_cmd = 'hhmake -i ../results/homolog_groups_expansion_and_collapse/results/jackhmmer_hits_alns/{0}.msa -M first -o {1} -name {0}'.format(group_tag, hhm_file).split(' ')
    subprocess.run(hhmake_cmd)

In [56]:
# create pairwise non redundant comparisons with itertools.comparisons
## list hhm files
hhm_list = [(hhm.split('/')[5].replace('.hhm', ''), hhm) for hhm in glob.glob('../results/homolog_groups_expansion_and_collapse/results/homolog_groups_HHMs/*hhm')]

# perform pairwise alingments with hhalign.
for pairwise_comp in itertools.combinations(hhm_list, 2):
  # create variable for each
  hhm_1 = pairwise_comp[0][1]
  hhm_group_1 = pairwise_comp[0][0]
  hhm_2 = pairwise_comp[1][1]
  hhm_group_2 = pairwise_comp[1][0]
  # run hhalign
  hhalign_out = '../results/homolog_groups_expansion_and_collapse/results/hhalign/outfiles/{0}_vs_{1}.hhr'.format(hhm_group_1, hhm_group_2)
  hhalign_tsv = '../results/homolog_groups_expansion_and_collapse/results/hhalign/tsv/{0}_vs_{1}.hhalign.tsv'.format(hhm_group_1, hhm_group_2)
  if not os.path.exists(hhalign_out):
    hhalign_cmd = 'hhalign -i {0} -t {1} -o {2} -atab {3}'.format(hhm_1, hhm_2, hhalign_out, hhalign_tsv).split(' ')
    subprocess.run(hhalign_cmd)

# creating dictionary with MSA sequences for each group, in order to posteriorly get alignment lengths (equal to HMM length used to filter)
seq_msas_length_dict = {}
for hg in glob.glob('../results/homolog_groups_expansion_and_collapse/results/homolog_groups_HMMs/*hmm'): 
    # get group name
    hg_tag = hg.split('/')[5].replace('.hmm', '')
    #print(hg_tag)
    with pyhmmer.plan7.HMMFile(hg) as hmm_file:
        hmm = next(hmm_file)
    hmm_len = len(hmm.consensus)
    seq_msas_length_dict.update({hg_tag: hmm_len})

In [57]:
# create function to parse HHR file and return a pandas data frame with the relevant info
def parse_hhsuite_hhr(hhr_file, group_1, group_2):
  try:
    handle_hhr = open(hhr_file, 'r')
    hhr_hhalign = HHsuiteIO.hhsuite2_text.Hhsuite2TextParser(handle = handle_hhr)
    hhr_hits = []
    # storing results in pandas dataframe
    for qresult in hhr_hhalign:
      for hit in qresult:
        for hsp in hit:
          hhr_hits.append(pd.DataFrame.from_dict({'Hit ID': [hit.id], 'Query ID': [hit.query_id], 'Alignment span': [hsp.aln_span], 'e-value': [hit.evalue]}))
    return pd.concat(hhr_hits)
  except:
    None

# running for example file
#example_hhr_table = parse_hhsuite_hhr(hhr_file = example_file, group_1 = '1233_LOEP01000012_1233', group_2 = '1385_LOEP01000012_1385')
# define auxiliart function
def get_coverage_hhr(hhr_table_row):
  hit_hmm = hhr_table_row['Hit ID'].to_list()[0]
  hit_hmm_length = seq_msas_length_dict[hit_hmm]
  query_hmm = hhr_table_row['Query ID'].to_list()[0]
  query_hmm_length = seq_msas_length_dict[query_hmm]
  aln_span = hhr_table_row['Alignment span'].to_list()[0]
  if hit_hmm_length >= query_hmm_length:
    # calculate coverage based on hit_hmm_length
    cov = aln_span/hit_hmm_length*100
  elif hit_hmm_length < query_hmm_length:
    cov = aln_span/query_hmm_length*100
  # return modified hhr table
  return hhr_table_row.assign(hit_hmm_length = hit_hmm_length, query_hmm_length = query_hmm_length, cov = cov)
    
#opa = example_hhr_table.assign(Hit_HMM_length = lambda x: seq_msas_length_dict[x['Hit ID'].to_list()[0]],
#                         Query_HMM_length = lambda x: seq_msas_length_dict[x['Query ID'].to_list()[0]])


In [58]:
if not os.path.exists('../results/homolog_groups_expansion_and_collapse/results/connected_groups/tsv/hhalign_filtered_table.tsv'):
    hhalign_results = []
    for hhr_file in glob.glob('../results/homolog_groups_expansion_and_collapse/results/hhalign/outfiles/*hhr'):
      # get group tags
      group_1_tag = hhr_file.split('/')[6].replace('.hhr', '').split('_vs_')[0]
      group_2_tag = hhr_file.split('/')[6].replace('.hhr', '').split('_vs_')[1]
      # getting coverage
      hhr_parsed = parse_hhsuite_hhr(hhr_file, group_1_tag, group_2_tag)
      try:
        hhr_parsed_with_cov = get_coverage_hhr(hhr_parsed)
      except:
        hhr_parsed_with_cov = None
      # parsing hhr file and appending result
      hhalign_results.append(hhr_parsed_with_cov)
    # concatenate to generate table
    hhalign_results_table = pd.concat(hhalign_results)
    hhalign_results_table.to_csv('../results/homolog_groups_expansion_and_collapse/results/connected_groups/tsv/hhalign_results_table.tsv', sep = '\t')
    # filtering table by evalue
    hhalign_results_table_filtered = hhalign_results_table.query("`e-value` < 1e-5 & cov >= 50")
    hhalign_results_table_filtered.to_csv('../results/homolog_groups_expansion_and_collapse/results/connected_groups/tsv/hhalign_filtered_table.tsv', sep = '\t')
elif os.path.exists('../results/homolog_groups_expansion_and_collapse/results/connected_groups/tsv/hhalign_filtered_table.tsv'):
    hhalign_results_table = pd.read_csv('../results/homolog_groups_expansion_and_collapse/results/connected_groups/tsv/hhalign_results_table.tsv', sep = '\t')
    hhalign_results_table_filtered = pd.read_csv('../results/homolog_groups_expansion_and_collapse/results/connected_groups/tsv/hhalign_filtered_table.tsv', sep = '\t')

In [59]:
hhalign_results_table_filtered.head()

,Unnamed: 0,Hit ID,Query ID,Alignment span,e-value,hit_hmm_length,query_hmm_length,cov
0,0,82926_Ga0207718_100100_978,41611_NZ_CP071597.1_384,343,2.300000e-126,317,330,103.939394
1,0,85158_Ga0207719_100190_476,82877_Ga0207718_100100_929,373,5.300000e-10,398,677,55.096012
2,0,37778_DRYJ01000053.1_300,36563_DQXS01000079.1_292,358,3.800000e-08,366,356,97.814208
3,0,41717_NZ_CP071597.1_490,45346_LOEP01000012_1256,366,1.300000e-19,322,343,106.705539
4,0,83636_Ga0207718_100100_1688,44950_LOEP01000012_860,163,6.300000e-09,253,227,64.426877


In [60]:
# creating graph and getting conected nodes (i.e. groups of interconected groups by HMM-HMM hits)
hmm_graph = nx.Graph()

# adding node for each group in table
hits_and_queries = []
for x in hhalign_results_table_filtered['Hit ID'].to_list():
  if not x in hits_and_queries:
    hits_and_queries.append(x)
    
for x in hhalign_results_table_filtered['Query ID'].to_list():
  if not x in hits_and_queries:
    hits_and_queries.append(x)
    
for group in hits_and_queries:
  # add node in graph
  hmm_graph.add_node(group)

# add edges between groups with hhalign significant HMM-HMM hits
for matches in zip(hhalign_results_table_filtered['Hit ID'].to_list(), hhalign_results_table_filtered['Query ID'].to_list()):
  # unpack variables
  node_1, node_2 = matches
  # add edge
  hmm_graph.add_edge(node_1, node_2)

# getting the connected components
components = nx.connected_components(hmm_graph)

In [61]:
# create dictionary with all ORF protein sequences; do the same for CDS sequences
# reading data frame with group data
orf_annotation = pd.read_csv('../results/MEs_predicted_orfs_renamed/data/annotation_table_ORFs_renamed.tsv', 
                             sep = '\t')
orf_annotation['Genome ID'] = orf_annotation['Genome ID'].astype(str)

# loading genome contigs
# listing genomes
genome_set = [(genome_file.split('/')[3].replace('ME.', '').replace('.fna', '').replace('.MAG', '').replace('.metagenome', ''),
               genome_file) for genome_file in glob.glob('../data/mobile_elements_sequences/*')]

# loading genomes with SeqIO.parse
genomes_dict = {}
for data_set in genome_set:
  # unpacking variables
  genome_tag, genome_file = data_set
  genome = SeqIO.parse(genome_file, 'fasta')
  # creating dictionary with sequences, descriptions and genome code
  genome_dict = {}
  for seq in genome:
    genome_dict.update({seq.id: (seq.description, seq.seq)})
  # updating dictionary
  genomes_dict.update({genome_tag: genome_dict})

# looping over DataFrame in order to get sequences
# iterating over genomes
# create dictionaries to store CDS and AA seqs
CDS_seq_dict = {}
AA_seq_dict = {}

for genome_id in [x[0] for x in genome_set]:
  # subsetting DataFrame to genomes with that ID
  subset_annot = orf_annotation.query("`Genome ID` == @genome_id")
  # creating variable to allocate all the sequence records for this case
  sequence_records = []
  sequence_records_aa = []
  # getting all sequences for that genome ID
  for orf_id, contig, strand, start, stop in zip(subset_annot['ORF ID'], subset_annot['contig'], subset_annot['strand'], subset_annot['START'], subset_annot['STOP']):
    # getting sequence
    contig_id = contig.split(' ')[0]
    if strand == '+':
     sequence = genomes_dict[genome_id][contig_id][1][start-1:stop]
    if strand == '-':
     sequence = genomes_dict[genome_id][contig_id][1][stop-1:start]
     sequence = sequence.reverse_complement()
    # creating SeqRecord object
    record = SeqRecord(sequence.upper(), id = orf_id, description = orf_id)
    sequence_records.append(record)
    # add to CDS_seq_dict
    CDS_seq_dict.update({orf_id: record})
    # add AA sequence to AA_seq_dict
    sequence_aa = sequence.translate(table = 11)
    record_aa = SeqRecord(sequence_aa.upper(), id = orf_id, description = orf_id)
    sequence_records_aa.append(record_aa)
    AA_seq_dict.update({orf_id: record_aa})

# create dictionary between old and new ORF IDs
orffeature2orfid_dict = {}
orfid2orffeature_dict = {}
for duple in zip(orf_annotation['getorf featured ID'].to_list(), orf_annotation['ORF ID'].to_list()):
  # depack duple
  feature_id, orf_id = duple
  # update dictionaries
  orffeature2orfid_dict.update({feature_id.split(' ')[0]: orf_id})
  orfid2orffeature_dict.update({orf_id: feature_id.split(' ')[0]})

# get hmmsearch results in order to retrieve hits for groups
hmmsearch_results = pd.read_csv('../results/homolog_groups_expansion_and_collapse/results/hmmsearch_results/hmmsearches_filtered_per_group.tsv', sep = '\t')
hmmsearch_results_grouped = hmmsearch_results.groupby('query_name')
# create dictionary and fill it
hmmsearch_results_per_group_dict = {}
for group_results in hmmsearch_results_grouped:
  # unpack variables
  group_tag, hmmsearch_table = group_results
  # get hits
  group_hits = hmmsearch_table.target_name.to_list()
  # translate hits
  group_hits = [orffeature2orfid_dict[hit] for hit in group_hits]
  # update dictionary
  hmmsearch_results_per_group_dict.update({group_tag: group_hits})
  
# get homologous groups and expand them with hmmsearch hits
homolog_groups_list = [(file.split('/')[3].replace('.faa', ''),
                        file) for file in glob.glob('../results/MEs_predicted_orfs_filtered_homologous_groups/*faa')]

homolog_groups_dict = {}
for set in homolog_groups_list:  
  # depack variables
  group_tag, group_file = set
  # append records to dictionary
  # get known records
  group_fasta = SeqIO.parse(group_file, 'fasta')
  group_fasta_list = [record for record in group_fasta]
  # get expanded records by searching in hmmsearch results and retrieving sequences from ORFs dictionary
  hitted_orfs_by_hmmsearch = hmmsearch_results_per_group_dict[group_tag]
  hmmsearch_AA_hits = [AA_seq_dict[hit] for hit in hitted_orfs_by_hmmsearch]
  ## backtranslate id to keep coherence in annotation # -> apparently not necessary in some cases, so performing conditional
  for record in hmmsearch_AA_hits:
    if record.id in list(orfid2orffeature_dict.keys()):
      record.id = orfid2orffeature_dict[record.id]
      record.description = ''
      record.name = ''
  # getting only those records not already present in group
  hmmsearch_AA_hits = [record for record in hmmsearch_AA_hits if not record.id in [record.id for record in group_fasta_list]]
  # append to list and update dictionary
  group_fasta_list.extend(hmmsearch_AA_hits)
  homolog_groups_dict.update({group_tag: group_fasta_list})

# define auxiliary function to get unique set of sequences from a list of records
def get_unique_record_set(list_of_records):
    # create empty list to allocate unique set of records
    unique_set_of_records = []
    # get over records and append the if they are not in the sequence set
    for target_record in list_of_records:
        # get actual set of records
        actual_set_of_records = [record.id for record in unique_set_of_records]
        if target_record.id not in actual_set_of_records:
            unique_set_of_records.append(target_record)
    # return <unique_set_of_records>
    return unique_set_of_records

# collapse the collapsable groups
collapsed_homolog_groups_list = [list(x) for x in components]
for duple in zip(range(1, len(collapsed_homolog_groups_list)+1), collapsed_homolog_groups_list):
  # depack variables
  i, groups = duple
  # create collapsed group name
  collapsed_group_tag = 'CG_{0}'.format(i)
  # getting sequences and appending them in same list all together
  collapsed_group_sequences = []
  for group in groups:
    # retrieve sequences
    group_seqs = homolog_groups_dict[group]
    # append to <collapsed_group_sequences>
    collapsed_group_sequences.extend(group_seqs)
    # pop the groups from <homolog_groups_dict>
    homolog_groups_dict.pop(group)
  # get unique set of sequences
  collapsed_group_sequences = get_unique_record_set(list_of_records = collapsed_group_sequences)
  # update with collapsed group sequences
  homolog_groups_dict.update({collapsed_group_tag: collapsed_group_sequences})

# translate names of sequences
for group in list(homolog_groups_dict.keys()):
  # get sequences and rename them
  sequences = homolog_groups_dict[group]
  for seq in sequences:
    # change seq.id and seq.description
    seq_id = seq.id
    #seq.id = orffeature2orfid_dict[seq_id]
    seq.description = ''
  # set the sequences in the dictionary
  homolog_groups_dict[group] = sequences
  
# create table with groups and sequences
# creating lists to allocate the data
group_tags = list(homolog_groups_dict.keys())
files = []
for tag in group_tags:
  # if isnt collapsed group, then add the word '_expanded'
  if re.match('^CG_', tag):
    files.append('../results/homolog_groups_expansion_and_collapse/results/connected_groups/fastas/protein/{0}.faa'.format(tag))
  else:
    files.append('../results/homolog_groups_expansion_and_collapse/results/connected_groups/fastas/protein/{0}_expanded.faa'.format(tag))

# getting sequence names and collapsing them in a line for the table
sequence_names = ['; '.join([seq.id for seq in records]) for records in list(homolog_groups_dict.values())]

# creating a pandas DataFrame
# creating dictionary to create a DataFrame object later
homolog_groups_data = {'Group tag': group_tags, 'File': files, 'Sequence names': sequence_names, 'Number of sequences': [len(x.split('; ')) for x in sequence_names]}
homolog_groups_table = pd.DataFrame.from_dict(homolog_groups_data, orient='columns')

# order table based on number of sequences
homolog_groups_table = homolog_groups_table.sort_values(by = ['Number of sequences'], axis = 0, ascending = False)

# export table
homolog_groups_table.to_csv('../results/homolog_groups_expansion_and_collapse/results/connected_groups/tsv/expanded_homologs_groups.tsv', sep = '\t')

# save FASTAs
for set in homolog_groups_table.groupby('Group tag'):
  # depack variables
  group_tag, row = set
  # get export file
  fasta_output = row.File.to_list()[0]
  # get sequences
  records = homolog_groups_dict[group_tag]
  # save records in FASTA file
  if not os.path.exists(fasta_output):
    with open(fasta_output, 'w') as output_handle:
        SeqIO.write(records, output_handle, 'fasta')

## Filtering for target PCGs, MAGs and metagenomes
Going to take into account only PCGs, MAGs and metagenomes of at least 20kbps.

In [62]:
# creating directory to allocate results
target_dirs = ['../results/homolog_groups_expansion_and_collapse/results/connected_groups_filtered',
               '../results/homolog_groups_expansion_and_collapse/results/connected_groups_filtered/fastas',
               '../results/homolog_groups_expansion_and_collapse/results/connected_groups_filtered/fastas/protein',
               '../results/homolog_groups_expansion_and_collapse/results/connected_groups_filtered/trees',
               '../results/homolog_groups_expansion_and_collapse/results/connected_groups_filtered/plots',
               '../results/homolog_groups_expansion_and_collapse/results/connected_groups_filtered/tsv']

for dir in target_dirs:
    create_dir(dir)

In [63]:
# getting all mobile sequences files and getting their length, then filtering to get target IDs
# create a dictionary between Genome IDs and taxonomy
genome2taxonomy_dict = {}
for index, row in genome2taxonomy_table.iterrows():
    # get genome and taxonomy
    genomeid = row['Genome_id'].replace('GCA_', '')
    taxonomy = row['Taxonomy']
    genome2taxonomy_dict.update({genomeid: taxonomy})

# create list to allocate rows with info, and later collapse into table
mobile_element_lengths_rows = []
for fasta_file in glob.glob('../data/mobile_elements_sequences/*fna'):
    #print(fasta_file)
    # getting group tag
    group_tag = fasta_file.split('/')[3].replace('ME.', '').replace('.MAG', '').replace('.metagenome', '').replace('.fna', '')
    # get a version with prefixes to sample ORFs
    me_orf_file = fasta_file.split('/')[3].replace('.fna', '.predicted_orfs.faa')
    #print(group_tag)
    # getting length in bp
    group_seq_lengths = [len(record.seq) for record in SeqIO.parse(fasta_file, 'fasta')]
    # append row
    mobile_element_lengths_rows.append(pd.DataFrame.from_dict({'Group tag': [group_tag], 'Taxonomy': [genome2taxonomy_dict[group_tag]], 'ME_file': [me_orf_file], 'Total length': [sum(group_seq_lengths)]}))

# concatenate into a table
mobile_element_lengths_table = pd.concat(mobile_element_lengths_rows)

# filtering table
mobile_element_lengths_table_filtered = mobile_element_lengths_table.query("`Total length` >= 20000")

# getting sequences from this organisms
target_organisms_ORFs = []
for index, row in mobile_element_lengths_table_filtered.iterrows():
    # get group tag
    group_tag = row['Group tag']
    # create ME orf file
    me_file = row['ME_file']
    me_file_full = '../results/MEs_predicted_orfs/{0}'.format(me_file)
    # get the sequences and store them
    seq_ids = [record.id for record in SeqIO.parse(me_file_full, 'fasta')]
    for seq_id in seq_ids:
        target_organisms_ORFs.append(seq_id)

# subsampling FASTA files of connected groups and saving sequences belonging to target groups
for fasta_file in glob.glob('../results/homolog_groups_expansion_and_collapse/results/connected_groups/fastas/protein/*faa'):
    # get new group direction
    new_group_fasta_file = fasta_file.replace('connected_groups', 'connected_groups_filtered')
    # get sequences
    group_seqs = [record for record in SeqIO.parse(fasta_file, 'fasta')]
    # filter sequences to get only those of target organisms
    filtered_group_seqs = [record for record in group_seqs if record.id in target_organisms_ORFs]
    # save in fasta file
    if not os.path.exists(new_group_fasta_file):
        with open(new_group_fasta_file, 'w') as handle_fasta:
            SeqIO.write(filtered_group_seqs, handle_fasta, 'fasta')

In [64]:
# saving target organisms table
mobile_element_lengths_table_filtered.to_csv('../results/target_organisms.tsv', sep ='\t')

In [65]:
mobile_element_lengths_table_filtered

,Group tag,Taxonomy,ME_file,Total length
0,ga0207733_100382,ga0207733_100382,ME.ga0207733_100382.metagenome.predicted_orfs.faa,30902
0,000830315.1,Nanoarchaeota_GW2011-AR20_sp000830315,ME.000830315.1.MAG.predicted_orfs.faa,829411
0,ga0222658_1000332,ga0222658_1000332,ME.ga0222658_1000332.metagenome.predicted_orfs...,26142
0,ga0116200_10000074,ga0116200_10000074,ME.ga0116200_10000074.metagenome.predicted_orf...,22669
0,ga0172379_10001592,ga0172379_10001592,ME.ga0172379_10001592.metagenome.predicted_orf...,44548
0,003644925.1,Fermentibacterota_Aegiribacteria_sp003644925,ME.003644925.1.MAG.predicted_orfs.faa,40684
0,60847.21,Halogeometricum_borinquense_strain_wsp4,ME.60847.21.predicted_orfs.faa,89772
0,ga0075122_10000827,ga0075122_10000827,ME.ga0075122_10000827.metagenome.predicted_orf...,23359
0,ga0172377_10000119,ga0172377_10000119,ME.ga0172377_10000119.metagenome.predicted_orf...,65850
0,ga0222667_1000160,ga0222667_1000160,ME.ga0222667_1000160.metagenome.predicted_orfs...,38591
